In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

ModuleNotFoundError: No module named 'pandas'

### Data Load

In [ ]:
# 데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}


for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        file_path = f"./data/{folder}/{split}_{suffix}.parquet"
        # 변수명 형식: {var_prefix}_{split}_{month}
        variable_name = f"{var_prefix}_{split}"
        globals()[variable_name] = pd.read_parquet(file_path, engine="pyarrow")
        print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_train is loaded from ./data/1.회원정보/train_회원정보.parquet
credit_train is loaded from ./data/2.신용정보/train_신용정보.parquet
sales_train is loaded from ./data/3.승인매출정보/train_승인매출정보.parquet
billing_train is loaded from ./data/4.청구입금정보/train_청구정보.parquet
balance_train is loaded from ./data/5.잔액정보/train_잔액정보.parquet
channel_train is loaded from ./data/6.채널정보/train_채널정보.parquet
marketing_train is loaded from ./data/7.마케팅정보/train_마케팅정보.parquet
performance_train is loaded from ./data/8.성과정보/train_성과정보.parquet
customer_test is loaded from ./data/1.회원정보/test_회원정보.parquet
credit_test is loaded from ./data/2.신용정보/test_신용정보.parquet
sales_test is loaded from ./data/3.승인매출정보/test_승인매출정보.parquet
billing_test is loaded from ./data/4.청구입금정보/test_청구정보.parquet
balance_test is loaded from ./data/5.잔액정보/test_잔액정보.parquet
channel_test is loaded from ./data/6.채널정보/test_채널정보.parquet
marketing_test is loaded from ./data/7.마케팅정보/test_마케팅정보.parquet
performance_test is loaded from ./data/8.성과정보/test_성과정보.parquet


0

### Data Preprocessing(1) : Concat & Merge

In [ ]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

In [ ]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train"]]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

customer_train_df is created with shape: (400000, 78)
credit_train_df is created with shape: (400000, 42)
sales_train_df is created with shape: (400000, 406)
billing_train_df is created with shape: (400000, 46)
balance_train_df is created with shape: (400000, 82)
channel_train_df is created with shape: (400000, 105)
marketing_train_df is created with shape: (400000, 64)
performance_train_df is created with shape: (400000, 49)


0

In [ ]:
#### Test ####

# test 데이터에 대해 train과 동일한 방법 적용
test_dfs = {}

for prefix in info_categories:
    df_list = [globals()[f"{prefix}_test"]]
    test_dfs[f"{prefix}_test_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_test_df is created with shape: {test_dfs[f'{prefix}_test_df'].shape}")


customer_test_df = test_dfs["customer_test_df"]
credit_test_df   = test_dfs["credit_test_df"]
sales_test_df    = test_dfs["sales_test_df"]
billing_test_df  = test_dfs["billing_test_df"]
balance_test_df  = test_dfs["balance_test_df"]
channel_test_df  = test_dfs["channel_test_df"]
marketing_test_df= test_dfs["marketing_test_df"]
performance_test_df = test_dfs["performance_test_df"]

gc.collect()

customer_test_df is created with shape: (100000, 77)
credit_test_df is created with shape: (400000, 42)
sales_test_df is created with shape: (400000, 406)
billing_test_df is created with shape: (400000, 46)
balance_test_df is created with shape: (400000, 82)
channel_test_df is created with shape: (400000, 105)
marketing_test_df is created with shape: (400000, 64)
performance_test_df is created with shape: (400000, 49)


0

In [ ]:
#### Train ####

train_df = customer_train_df.merge(credit_train_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: train_step1, shape:", train_df.shape)
del customer_train_df, credit_train_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장 
merge_list = [
    ("sales_train_df",    "Step2"),
    ("billing_train_df",  "Step3"),
    ("balance_train_df",  "Step4"),
    ("channel_train_df",  "Step5"),
    ("marketing_train_df","Step6"),
    ("performance_train_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    train_df = train_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: train_{step}, shape:", train_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: train_step1, shape: (400000, 118)
Step2 저장 완료: train_Step2, shape: (400000, 522)
Step3 저장 완료: train_Step3, shape: (400000, 566)
Step4 저장 완료: train_Step4, shape: (400000, 646)
Step5 저장 완료: train_Step5, shape: (400000, 749)
Step6 저장 완료: train_Step6, shape: (400000, 811)
최종 저장 완료: train_최종, shape: (400000, 858)


In [ ]:
#### Test ####

test_df = customer_test_df.merge(credit_test_df, on=['기준년월', 'ID'], how='left')
print("Step1 저장 완료: test_step1, shape:", test_df.shape)
del customer_test_df, credit_test_df
gc.collect()

# 이후 merge할 데이터프레임 이름과 단계 정보를 리스트에 저장
merge_list = [
    ("sales_test_df",    "Step2"),
    ("billing_test_df",  "Step3"),
    ("balance_test_df",  "Step4"),
    ("channel_test_df",  "Step5"),
    ("marketing_test_df","Step6"),
    ("performance_test_df", "최종")
]

# 나머지 단계 merge
for df_name, step in merge_list:
    # globals()로 동적 변수 접근하여 merge 수행
    test_df = test_df.merge(globals()[df_name], on=['기준년월', 'ID'], how='left')
    print(f"{step} 저장 완료: test_{step}, shape:", test_df.shape)
    # 사용한 변수는 메모리 해제를 위해 삭제
    del globals()[df_name]
    gc.collect()

Step1 저장 완료: test_step1, shape: (100000, 117)
Step2 저장 완료: test_Step2, shape: (100000, 521)
Step3 저장 완료: test_Step3, shape: (100000, 565)
Step4 저장 완료: test_Step4, shape: (100000, 645)
Step5 저장 완료: test_Step5, shape: (100000, 748)
Step6 저장 완료: test_Step6, shape: (100000, 810)
최종 저장 완료: test_최종, shape: (100000, 857)


### Data Preprocessing(2) : Encoding

In [ ]:
feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [ ]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [ ]:
gc.collect()

0

### Train

In [ ]:
try:
    model = xgb.XGBClassifier(
        tree_method='gpu_hist',  # GPU 모드 설정
        gpu_id=0,
        random_state=42
    )
    print("GPU 사용 가능: gpu_hist 모드 적용")
    model.fit(X, y_encoded)
    
except Exception:
    model = xgb.XGBClassifier(
        random_state=42
    )
    print("GPU 사용 불가: CPU 모드 적용")
    model.fit(X, y_encoded)

GPU 사용 가능: gpu_hist 모드 적용


GPU 사용 불가: CPU 모드 적용


In [ ]:
model.fit(X, y_encoded)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


### Predict

In [ ]:
X_test.drop(columns=['ID'],inplace=True)

In [ ]:
# row-level 예측 수행
y_test_pred = model.predict(X_test)
# 예측 결과를 변환
y_test_pred_labels = le_target.inverse_transform(y_test_pred)

# row 단위 예측 결과를 test_data에 추가
test_data = test_df.copy()  # 원본 유지
test_data["pred_label"] = y_test_pred_labels

### Submission

In [ ]:
submission = test_data.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

In [ ]:
submission.to_csv('./base_submit.csv',index=False)

In [ ]:
importance = model.get_booster().get_score(importance_type='weight')
importance_df = pd.DataFrame(importance.items(), columns=['feature', 'importance'])\
    .sort_values('importance', ascending=False)

print(importance_df)
print(y)



           feature  importance
121  이용금액_일시불_R12M       591.0
60        월상환론한도금액       480.0
7          입회일자_신용       461.0
127   이용금액_체크_R12M       396.0
382     정상청구원금_B5M       391.0
..             ...         ...
442       연체일자_B0M         1.0
443       연체잔액_B0M         1.0
368   이용개월수_전체_R6M         1.0
497   인입횟수_ARS_B0M         1.0
344   이용금액_D페이_R3M         1.0

[608 rows x 2 columns]
0         D
1         E
2         C
3         D
4         E
         ..
399995    E
399996    D
399997    C
399998    E
399999    E
Name: Segment, Length: 400000, dtype: object
